In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import iri_dataset_generator as iri

# Description

This is a test model to figure out standardized training and dataset formatting

## Preprocessing

In [ ]:
SEQUENCE_LENGTH = 10
train, test = iri.load_iri_datasets(path="../training_data/IRI-only.parquet",seq_length=SEQUENCE_LENGTH)

## Model Definition

In [ ]:
class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.f1 = nn.Linear(SEQUENCE_LENGTH * 3, 128)
        self.f2 = nn.Linear(128, 128)
        self.f4 = nn.Linear(128, 128)
        self.f3 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = x.view(-1, 30)
        x = self.f1(x)
        x = self.f2(x)
        x = self.f4(x)
        x = self.f3(x)
        return x
    
print(train[0])

## Training

In [ ]:
model = FNN()
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, train, test, loss, optimizer, epochs=20, batch_size=2048, test_every_n=1):
    train_data = DataLoader(train, batch_size=batch_size, shuffle=True)
    bar = tqdm(range(epochs), desc="Training Epoch")
    test_loss=0
    for i in bar:
        avgloss = 0
        num_losses = 0
        for _, data in enumerate(train_data):
            inputs, goal = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            computed_loss=loss(model(inputs), goal)
            computed_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

            num_losses += 1
            avgloss += computed_loss.item()
        if i % test_every_n == 0:
            test_data = DataLoader(test, batch_size=batch_size, shuffle=True)
            test_losss = 0
            test_num_losses = 0
            for _, data in enumerate(test_data):
                inputs, goal = data[0].to(device), data[1].to(device)
                test_losss += loss(model(inputs), goal).item()
                test_num_losses += 1
            test_loss = test_losss/test_num_losses
        bar.set_postfix({"Train Loss": avgloss/num_losses, "Test Loss": test_loss})

model.to(device)
train_model(model, train, test, loss, optimizer, epochs=200, test_every_n=1)
        